In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# 各月新增对比

In [2]:
#关联库
from pyecharts import Map
from pyecharts import online
from pyecharts import Page
from pyecharts import Grid
from pyecharts import Bar
from pyecharts import Line
import pandas as pd
import numpy as np
import Levenshtein as lv
#from pyecharts import configure

In [3]:
#Q1Q2注册及暑假活跃
#原始数据 载入 清洗
zc1_6_hy7_8 = pd.read_csv("zc1_6_hy7_8.csv")
zc1_6_hy7_8.columns = ['c', 'b','a']
zc1_6_hy7_8=zc1_6_hy7_8.ix[:,['a', 'b','c']]
df=zc1_6_hy7_8['c'].str.split(' ', 2, True)
df=pd.concat([zc1_6_hy7_8.ix[:,['a', 'b']], df], axis=1)
df.columns = ['活跃','注册','省','市','区']
dfpvt2=pd.pivot_table(df,index=["省"],aggfunc=np.sum)
dfpvt2.eval('留存 = 活跃 / 注册', inplace=True)
cleanlist=['广东省东莞市大立学校','null','河南省周口市沈丘县享阅小学','浙江省宁波市市辖区国家高新区实验学校国家高新区实验学校','暂无','未选择学校']
dfpvt2=dfpvt2.drop(cleanlist)
dfpvt2=dfpvt2.dropna()

#图表数据提取
pro = ["北京","天津","黑龙江","吉林","辽宁","河北","河南","山东","山西","陕西","内蒙古","宁夏","甘肃","新疆","青海","西藏","湖北","安徽","江苏","上海","浙江","福建","湖南","江西","四川","重庆","贵州","云南","广东","广西","海南","香港","澳门","台湾"]
ro = [] 
for i in list(dfpvt2.index):
    for j in pro:
        if lv.ratio(i,j) !=0:
            ro.append([i,j,lv.ratio(i,j)])
ro=pd.DataFrame(ro,columns=['原数据省','标准省','ratio'])
ro=ro.drop_duplicates()
ro=ro.sort_values(['原数据省','ratio'],ascending=[1,0])
rankcolumn=ro.groupby('原数据省').rank(method='first',numeric_only=True,ascending=False)
ro['rankcolumn']=rankcolumn
ro=ro[ro.rankcolumn==1]
ro.reindex()
comb=list(zip(list(ro['原数据省']),list(ro['标准省'])))
org_prov=list(dfpvt2.index)
for i in  range(len(org_prov)):
        for j in comb:
               if org_prov[i]==j[0]:
                    org_prov[i]=(j[1])

In [4]:
#绘图 上半年注册 map
value1 = list(dfpvt2['注册'])
attr1 = org_prov
map1 = Map("上半年注册量", width=1000, height=1000)
map1.add(
    "",
    attr1,
    value1,
    maptype="china",
    is_visualmap=True,
    is_label_show=True,
    visual_text_color="#000",
    visual_range = [0,250000],
    is_more_utils=True,
    is_piecewise=True,
    pieces=[
             { "min": 200001, "label": "成绩斐然"},
             {"max": 200000, "min": 50001, "label": "卓有成效"},
             {"max": 50000, "min": 10001, "label": "已有起色"},
             {"max": 10000, "min": 1001, "label": "发展初期"},
             {"max": 1000, "min": 0, "label": "起步阶段"},
         ])
online()
map1

In [5]:
#绘图 Q1Q2用户暑假留存
value2 = [i*100 for i in list(dfpvt2['留存'])]
attr2 = org_prov
map2 = Map("上半年注册暑假留存率", width=1000, height=1000)
map2.add(
    "",
    attr2,
    value2,
    maptype="china",
    is_visualmap=True,
    is_label_show=True,
    visual_text_color="#000",
    visual_range = [0,100],
    is_more_utils=True,
#    is_piecewise=True,
#     pieces=[
#             {"max": 250000, "min": 150001, "label": "成绩斐然"},
#             {"max": 150000, "min": 50001, "label": "卓有成效"},
#             {"max": 50000, "min": 10001, "label": "已有起色"},
#             {"max": 10000, "min": 1001, "label": "发展初期"},
#             {"max": 1000, "min": 0, "label": "起步阶段"},
#         ]
        )
online()
map2

In [7]:
#Q3注册
#原始数据 载入 清洗
Q3zc = pd.read_csv("Q3zc.csv")
Q3zc.columns = ['b','a']
Q3zc=Q3zc.ix[:,['a', 'b']]
df_tmp=Q3zc['b'].str.split(' ', 2, True)
Q3zc=pd.concat([Q3zc.ix[:,'a'], df_tmp], axis=1)
Q3zc.columns = ['注册','省','市','区']
Q3zcpvt=pd.pivot_table(Q3zc,index=["省"],aggfunc=np.sum)
#dfpvt2.eval('留存 = 活跃 / 注册', inplace=True)
cleanlist=['null','暂无','未选择学校']
Q3zcpvt=Q3zcpvt.drop(cleanlist)
Q3zcpvt=Q3zcpvt.dropna()

#图表数据提取
pro = ["北京","天津","黑龙江","吉林","辽宁","河北","河南","山东","山西","陕西","内蒙古","宁夏","甘肃","新疆","青海","西藏","湖北","安徽","江苏","上海","浙江","福建","湖南","江西","四川","重庆","贵州","云南","广东","广西","海南","香港","澳门","台湾"]
ro = [] 
for i in list(Q3zcpvt.index):
    for j in pro:
        if lv.ratio(i,j) !=0:
            ro.append([i,j,lv.ratio(i,j)])
ro=pd.DataFrame(ro,columns=['原数据省','标准省','ratio'])
ro=ro.drop_duplicates()
ro=ro.sort_values(['原数据省','ratio'],ascending=[1,0])
rankcolumn=ro.groupby('原数据省').rank(method='first',numeric_only=True,ascending=False)
ro['rankcolumn']=rankcolumn
ro=ro[ro.rankcolumn==1]
ro.reindex()
comb=list(zip(list(ro['原数据省']),list(ro['标准省'])))
org_prov=list(Q3zcpvt.index)
for i in  range(len(org_prov)):
        for j in comb:
               if org_prov[i]==j[0]:
                    org_prov[i]=(j[1])

In [8]:
#绘图 Q3注册
value3 = list(Q3zcpvt['注册'])
attr3 = org_prov
map3 = Map("Q3注册", width=1000, height=1000)
map3.add(
    "",
    attr3,
    value3,
    maptype="china",
    is_visualmap=True,
    is_label_show=True,
    visual_text_color="#000",
    visual_range = [0,200000],
    is_more_utils=True,
    is_piecewise=True,
    pieces=[
             { "min": 200001, "label": "成绩斐然"},
             {"max": 200000, "min": 50001, "label": "卓有成效"},
             {"max": 50000, "min": 10001, "label": "已有起色"},
             {"max": 10000, "min": 1001, "label": "发展初期"},
             {"max": 1000, "min": 0, "label": "起步阶段"},
         ]
        )
online()
map3

In [9]:
#定义sort key函数
def sortByName(object_one):
    return object_one[0]

In [10]:
#以Q12注册排序省份
zipped=list(zip(value1,attr1,value3,attr3))
zipped.sort(key=sortByName,reverse=True)
v1,a1,v3,a3=zip(*zipped)
#绘图 Q1Q2\Q3注册 Bar
bar1 = Bar("截至Q3注册",width=1000, height=800)
bar1.add("Q1Q2",a1,v1,is_stack=True,mark_line=["average"], mark_point=["max", "min"])
bar1.add("Q3",a3,v3,is_stack=True,mark_line=["average"], mark_point=["max", "min"],is_more_utils=True,is_datazoom_show=True)

In [11]:
#Q123 日新增
org_ios_source={"code":10001,"msg":"Request success","app_data":{"x_axis":["2018-01-01","2018-01-02","2018-01-03","2018-01-04","2018-01-05","2018-01-06","2018-01-07","2018-01-08","2018-01-09","2018-01-10","2018-01-11","2018-01-12","2018-01-13","2018-01-14","2018-01-15","2018-01-16","2018-01-17","2018-01-18","2018-01-19","2018-01-20","2018-01-21","2018-01-22","2018-01-23","2018-01-24","2018-01-25","2018-01-26","2018-01-27","2018-01-28","2018-01-29","2018-01-30","2018-01-31","2018-02-01","2018-02-02","2018-02-03","2018-02-04","2018-02-05","2018-02-06","2018-02-07","2018-02-08","2018-02-09","2018-02-10","2018-02-11","2018-02-12","2018-02-13","2018-02-14","2018-02-15","2018-02-16","2018-02-17","2018-02-18","2018-02-19","2018-02-20","2018-02-21","2018-02-22","2018-02-23","2018-02-24","2018-02-25","2018-02-26","2018-02-27","2018-02-28","2018-03-01","2018-03-02","2018-03-03","2018-03-04","2018-03-05","2018-03-06","2018-03-07","2018-03-08","2018-03-09","2018-03-10","2018-03-11","2018-03-12","2018-03-13","2018-03-14","2018-03-15","2018-03-16","2018-03-17","2018-03-18","2018-03-19","2018-03-20","2018-03-21","2018-03-22","2018-03-23","2018-03-24","2018-03-25","2018-03-26","2018-03-27","2018-03-28","2018-03-29","2018-03-30","2018-03-31","2018-04-01","2018-04-02","2018-04-03","2018-04-04","2018-04-05","2018-04-06","2018-04-07","2018-04-08","2018-04-09","2018-04-10","2018-04-11","2018-04-12","2018-04-13","2018-04-14","2018-04-15","2018-04-16","2018-04-17","2018-04-18","2018-04-19","2018-04-20","2018-04-21","2018-04-22","2018-04-23","2018-04-24","2018-04-25","2018-04-26","2018-04-27","2018-04-28","2018-04-29","2018-04-30","2018-05-01","2018-05-02","2018-05-03","2018-05-04","2018-05-05","2018-05-06","2018-05-07","2018-05-08","2018-05-09","2018-05-10","2018-05-11","2018-05-12","2018-05-13","2018-05-14","2018-05-15","2018-05-16","2018-05-17","2018-05-18","2018-05-19","2018-05-20","2018-05-21","2018-05-22","2018-05-23","2018-05-24","2018-05-25","2018-05-26","2018-05-27","2018-05-28","2018-05-29","2018-05-30","2018-05-31","2018-06-01","2018-06-02","2018-06-03","2018-06-04","2018-06-05","2018-06-06","2018-06-07","2018-06-08","2018-06-09","2018-06-10","2018-06-11","2018-06-12","2018-06-13","2018-06-14","2018-06-15","2018-06-16","2018-06-17","2018-06-18","2018-06-19","2018-06-20","2018-06-21","2018-06-22","2018-06-23","2018-06-24","2018-06-25","2018-06-26","2018-06-27","2018-06-28","2018-06-29","2018-06-30","2018-07-01","2018-07-02","2018-07-03","2018-07-04","2018-07-05","2018-07-06","2018-07-07","2018-07-08","2018-07-09","2018-07-10","2018-07-11","2018-07-12","2018-07-13","2018-07-14","2018-07-15","2018-07-16","2018-07-17","2018-07-18","2018-07-19","2018-07-20","2018-07-21","2018-07-22","2018-07-23","2018-07-24","2018-07-25","2018-07-26","2018-07-27","2018-07-28","2018-07-29","2018-07-30","2018-07-31","2018-08-01","2018-08-02","2018-08-03","2018-08-04","2018-08-05","2018-08-06","2018-08-07","2018-08-08","2018-08-09","2018-08-10","2018-08-11","2018-08-12","2018-08-13","2018-08-14","2018-08-15","2018-08-16","2018-08-17","2018-08-18","2018-08-19","2018-08-20","2018-08-21","2018-08-22","2018-08-23","2018-08-24","2018-08-25","2018-08-26","2018-08-27","2018-08-28","2018-08-29","2018-08-30","2018-08-31","2018-09-01","2018-09-02","2018-09-03","2018-09-04","2018-09-05","2018-09-06","2018-09-07","2018-09-08","2018-09-09","2018-09-10","2018-09-11","2018-09-12","2018-09-13","2018-09-14","2018-09-15","2018-09-16","2018-09-17","2018-09-18","2018-09-19","2018-09-20","2018-09-21","2018-09-22","2018-09-23","2018-09-24","2018-09-25","2018-09-26"],"series":[{"names":["新增用户"],"original_names":["新增用户"],"values":[0,1,4,7,13,15,1,4,8,7,764,1038,738,549,333,330,280,276,270,265,240,166,252,335,650,593,375,283,180,429,762,1161,1083,700,697,588,421,408,280,308,270,235,208,174,170,103,119,102,141,118,147,168,214,181,217,170,189,175,131,176,176,151,131,175,201,199,156,181,212,163,158,132,155,131,122,109,108,194,246,409,421,774,356,232,518,637,755,362,356,418,271,510,1418,1379,738,475,485,1119,706,838,1085,1581,974,425,413,1310,1441,1211,918,940,554,693,1762,1772,1674,1688,1347,844,624,617,567,771,1162,2653,1053,884,1567,1490,1921,1346,2151,897,722,1244,2360,2267,1224,1034,941,768,1220,2164,2492,3518,2593,1553,1285,1870,2903,4048,2310,1466,1868,1541,3957,4979,3771,3564,3470,1840,1648,3046,5567,7570,10533,8724,4917,3473,2637,10679,11398,11968,8780,6823,4559,8762,8975,8140,7560,6511,5445,3903,7572,7950,7474,6307,6403,3769,3315,4487,4094,3694,3181,2895,2548,2100,2617,2815,2614,2104,2003,1686,1657,1688,1836,2919,2496,2371,1791,1737,1849,1889,1520,1453,1357,1127,1100,1308,1239,1118,1119,1325,968,945,1132,1058,956,1021,945,794,790,977,1216,1152,1088,915,694,690,825,822,920,955,1070,735,899,987,1086,1280,1423,5079,2784,2087,3266,3015,3283,3643,2148,1446,1352,2235,3184,3578,3463,2905,1913,1506,1344,3172,3294]}]}}
org_android_source={"code":10001,"msg":"Request success","app_data":{"x_axis":["2018-01-01","2018-01-02","2018-01-03","2018-01-04","2018-01-05","2018-01-06","2018-01-07","2018-01-08","2018-01-09","2018-01-10","2018-01-11","2018-01-12","2018-01-13","2018-01-14","2018-01-15","2018-01-16","2018-01-17","2018-01-18","2018-01-19","2018-01-20","2018-01-21","2018-01-22","2018-01-23","2018-01-24","2018-01-25","2018-01-26","2018-01-27","2018-01-28","2018-01-29","2018-01-30","2018-01-31","2018-02-01","2018-02-02","2018-02-03","2018-02-04","2018-02-05","2018-02-06","2018-02-07","2018-02-08","2018-02-09","2018-02-10","2018-02-11","2018-02-12","2018-02-13","2018-02-14","2018-02-15","2018-02-16","2018-02-17","2018-02-18","2018-02-19","2018-02-20","2018-02-21","2018-02-22","2018-02-23","2018-02-24","2018-02-25","2018-02-26","2018-02-27","2018-02-28","2018-03-01","2018-03-02","2018-03-03","2018-03-04","2018-03-05","2018-03-06","2018-03-07","2018-03-08","2018-03-09","2018-03-10","2018-03-11","2018-03-12","2018-03-13","2018-03-14","2018-03-15","2018-03-16","2018-03-17","2018-03-18","2018-03-19","2018-03-20","2018-03-21","2018-03-22","2018-03-23","2018-03-24","2018-03-25","2018-03-26","2018-03-27","2018-03-28","2018-03-29","2018-03-30","2018-03-31","2018-04-01","2018-04-02","2018-04-03","2018-04-04","2018-04-05","2018-04-06","2018-04-07","2018-04-08","2018-04-09","2018-04-10","2018-04-11","2018-04-12","2018-04-13","2018-04-14","2018-04-15","2018-04-16","2018-04-17","2018-04-18","2018-04-19","2018-04-20","2018-04-21","2018-04-22","2018-04-23","2018-04-24","2018-04-25","2018-04-26","2018-04-27","2018-04-28","2018-04-29","2018-04-30","2018-05-01","2018-05-02","2018-05-03","2018-05-04","2018-05-05","2018-05-06","2018-05-07","2018-05-08","2018-05-09","2018-05-10","2018-05-11","2018-05-12","2018-05-13","2018-05-14","2018-05-15","2018-05-16","2018-05-17","2018-05-18","2018-05-19","2018-05-20","2018-05-21","2018-05-22","2018-05-23","2018-05-24","2018-05-25","2018-05-26","2018-05-27","2018-05-28","2018-05-29","2018-05-30","2018-05-31","2018-06-01","2018-06-02","2018-06-03","2018-06-04","2018-06-05","2018-06-06","2018-06-07","2018-06-08","2018-06-09","2018-06-10","2018-06-11","2018-06-12","2018-06-13","2018-06-14","2018-06-15","2018-06-16","2018-06-17","2018-06-18","2018-06-19","2018-06-20","2018-06-21","2018-06-22","2018-06-23","2018-06-24","2018-06-25","2018-06-26","2018-06-27","2018-06-28","2018-06-29","2018-06-30","2018-07-01","2018-07-02","2018-07-03","2018-07-04","2018-07-05","2018-07-06","2018-07-07","2018-07-08","2018-07-09","2018-07-10","2018-07-11","2018-07-12","2018-07-13","2018-07-14","2018-07-15","2018-07-16","2018-07-17","2018-07-18","2018-07-19","2018-07-20","2018-07-21","2018-07-22","2018-07-23","2018-07-24","2018-07-25","2018-07-26","2018-07-27","2018-07-28","2018-07-29","2018-07-30","2018-07-31","2018-08-01","2018-08-02","2018-08-03","2018-08-04","2018-08-05","2018-08-06","2018-08-07","2018-08-08","2018-08-09","2018-08-10","2018-08-11","2018-08-12","2018-08-13","2018-08-14","2018-08-15","2018-08-16","2018-08-17","2018-08-18","2018-08-19","2018-08-20","2018-08-21","2018-08-22","2018-08-23","2018-08-24","2018-08-25","2018-08-26","2018-08-27","2018-08-28","2018-08-29","2018-08-30","2018-08-31","2018-09-01","2018-09-02","2018-09-03","2018-09-04","2018-09-05","2018-09-06","2018-09-07","2018-09-08","2018-09-09","2018-09-10","2018-09-11","2018-09-12","2018-09-13","2018-09-14","2018-09-15","2018-09-16","2018-09-17","2018-09-18","2018-09-19","2018-09-20","2018-09-21","2018-09-22","2018-09-23","2018-09-24","2018-09-25","2018-09-26"],"series":[{"names":["新增用户"],"original_names":["新增用户"],"values":[13,20,4,18,18,4,1,11,3,128,4936,2685,2363,1952,1128,1064,1838,1042,960,922,881,593,811,1380,1838,2288,1613,1229,833,1041,802,2985,2064,1329,1574,1342,768,603,674,499,443,478,373,345,289,196,198,232,268,246,289,318,365,348,338,325,289,313,265,408,384,357,368,382,373,314,228,337,365,314,515,466,414,426,455,570,530,375,735,1356,1671,3200,1266,952,1480,3148,2337,1228,1028,1044,768,1476,3025,3783,1773,998,899,2403,2397,4079,4817,5036,2683,1149,942,5208,5857,4697,3834,3930,1831,1874,3594,6246,5215,5626,4213,3254,2283,1574,1266,2777,4110,10077,3456,2645,5693,4865,6702,3729,5710,2693,1901,3864,8100,10410,5407,3948,2907,2003,4819,7601,10610,10282,12009,6292,4086,8404,11777,16863,10421,5977,5282,4608,17299,20786,12784,12997,14826,6104,4451,11706,25423,55542,62986,55210,32777,20542,13750,66779,93749,99781,85136,49765,30835,67768,68122,56506,52200,44261,31445,22976,52307,55217,51679,39454,38131,20873,16008,20740,22558,19253,16619,16973,15229,10232,12965,14108,11687,8396,6950,4529,3635,3411,4949,21644,18622,17009,12991,11565,10861,10948,5305,4515,3699,3580,3220,2834,2449,2341,2119,2323,2067,2126,1978,1960,1807,1777,1715,1811,1917,1762,2213,1816,1707,1772,1678,1843,1819,1721,2218,2818,2762,2449,2568,2585,3424,4587,4784,21914,10073,7147,11181,10455,10255,14306,8963,5840,4637,7555,11478,14306,15034,13569,7179,4398,3639,11237,14262]}]}}
#获取key_value
ios_day_timeline = org_ios_source['app_data']['x_axis']
ios_day_register_value = org_ios_source['app_data']['series'][0]['values']
android_day_timeline = org_android_source['app_data']['x_axis']
android_day_register_value = org_android_source['app_data']['series'][0]['values']
#折线图
line1 = Line("Q123 日新增",width=1020, height=800)
line1.add(
    "ios",
    ios_day_timeline,ios_day_register_value,
    mark_point=["max", "min"],
    mark_line=["average"],mark_point_symbolsize=100
)
line1.add(
    "adnroid",
    android_day_timeline,android_day_register_value,
    mark_point=["max", "min"],
    mark_line=["average"],
    yaxis_formatter="人",is_more_utils=True,is_datazoom_show=True,yaxis_max =120000,mark_point_symbolsize=100
)
line1